In [1]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import pandas as pd
import numpy as np
import sources

In [2]:
cfg = sources.default_config()
mlclient = MlflowClient(cfg.mlflow.uri)

In [3]:
EXP = "SteeredCNN-AV"
exp = mlclient.get_experiment_by_name(EXP)


In [4]:
def parse_runs(runs):
    data = {**{('run', 'exp'): [],
               ('run', 'sub'): [],
               ('run', 'subexp'): []}}
    
    metrics = set()
    params = set()
    
    for i, r in enumerate(runs):
        data[('run', 'exp')] += [r.data.tags.get('exp', '-')]
        data[('run', 'sub')] += [r.data.tags.get('sub', '-')]
        data[('run', 'subexp')] += [r.data.tags.get('subexp', '-')]
        
        for k, v in r.data.metrics.items():
            if ('metrics', k) not in data:
                data[('metrics', k)] = ['-']*i
            data[('metrics', k)] += [v]
            metrics.update([k])
        for k in metrics.difference(r.data.metrics.keys()):
            data[('metrics', k)] += ['-']
            
        for k, v in r.data.params.items():
            if ('params', k) not in data:
                data[('params', k)] = ['-']*i
            data[('params', k)] += [v]
            params.update([k])
        for k in params.difference(r.data.params.keys()):
            data[('params', k)] += ['-']
            
    df = pd.DataFrame(data)
    df.columns.names = ['category', 'item']
    return df

def fetch_runs(exp, filter=None):
    if isinstance(exp, str):
        exp = mlclient.get_experiment_by_name(EXP)
    runs = mlclient.list_run_infos(exp.experiment_id, ViewType.ACTIVE_ONLY)
    runs = [mlclient.get_run(r.run_id) for r in runs if r.status=='FINISHED']
    
    if filter is not None:
        runs = [_ for _ in runs if filter(_)]
    
    return parse_runs(runs)

In [5]:
d = fetch_runs(exp, lambda r: r.data.tags.get('subexp', '-').startswith('[ParamStarve]'))
del d['params']
for m in list(d['metrics'].columns):
    if not m.startswith(('MESS','HRF','DRI')) and m!='best-val-acc-epoch':
        del d[('metrics', m)]


In [6]:
g = (d.sort_values(('metrics', 'DRIVE-acc'), ascending=False)
      .groupby(('run', 'subexp')).head(3)
      .groupby(('run', 'subexp')).agg(['mean','std'])
      .sort_values(('metrics','DRIVE-acc','mean'), ascending=False) )

In [7]:
g[('metrics','DRIVE-acc')].apply(lambda r: f"{r['mean']*100:.1f} ± {r['std']:.1%}", axis=1)

/home/gaby/.conda/envs/nnet/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2878: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)


(run, subexp)
[ParamStarve] 2-22    83.3 ± 0.3%
[ParamStarve] 2-6     83.2 ± 0.3%
[ParamStarve] 2-11    82.9 ± 0.3%
[ParamStarve] 1-11    82.5 ± 0.2%
[ParamStarve] 1-22    82.3 ± 0.0%
[ParamStarve] 1-6     81.1 ± 0.9%
[ParamStarve] 3-22    79.8 ± 5.4%
[ParamStarve] 3-11    79.7 ± 2.3%
[ParamStarve] 2-2     79.5 ± 0.1%
[ParamStarve] 2-3     78.0 ± 1.4%
[ParamStarve] 3-6     77.1 ± 0.7%
[ParamStarve] 1-2     76.4 ± 1.0%
[ParamStarve] 3-3     75.1 ± 1.2%
[ParamStarve] 3-2     74.3 ± 0.8%
dtype: object

In [11]:
g

category             metrics                                          \
item               DRIVE-acc           DRIVE-roc           DRIVE-iou   
                        mean       std      mean       std      mean   
(run, subexp)                                                          
[ParamStarve] 2-22  0.833218  0.002657  0.916837  0.006602  0.713817   
[ParamStarve] 2-6   0.832172  0.003275  0.916030  0.007735  0.712462   
[ParamStarve] 2-11  0.828654  0.003259  0.910299  0.002726  0.707247   
[ParamStarve] 1-11  0.825472  0.002203  0.917045  0.004298  0.702756   
[ParamStarve] 1-22  0.822958  0.000382  0.907702  0.005476  0.699003   
[ParamStarve] 1-6   0.811301  0.009411  0.905211  0.008575  0.682459   
[ParamStarve] 3-22  0.798214  0.054441  0.879372  0.055868  0.666233   
[ParamStarve] 3-11  0.797321  0.023455  0.887221  0.014794  0.662814   
[ParamStarve] 2-2   0.795334  0.000936  0.877783  0.008538  0.660160   
[ParamStarve] 2-3   0.780030  0.013838  0.862085  0.021002  0.638727   
[ParamStarve] 3-6   0.771142  0.006800  0.847164  0.011803  0.626958   
[ParamStarve] 1-2   0.763781  0.009606  0.845942  0.010019  0.617867   
[ParamStarve] 3-3   0.750881  0.011906  0.830058  0.008560  0.600992   
[ParamStarve] 3-2   0.742876  0.007920  0.813210  0.011028  0.590086   

category                                                              \
item                           HRF-roc             HRF-iou             
                         std      mean       std      mean       std   
(run, subexp)                                                          
[ParamStarve] 2-22  0.004128  0.791524  0.034887  0.547700  0.042990   
[ParamStarve] 2-6   0.004953  0.828487  0.013967  0.583525  0.018697   
[ParamStarve] 2-11  0.004857  0.820361  0.030925  0.573606  0.044881   
[ParamStarve] 1-11  0.003181  0.841688  0.014330  0.583555  0.015047   
[ParamStarve] 1-22  0.000557  0.797780  0.006207  0.554812  0.001304   
[ParamStarve] 1-6   0.013177  0.801590  0.002079  0.539825  0.007899   
[ParamStarve] 3-22  0.074582  0.815970  0.028839  0.577953  0.031122   
[ParamStarve] 3-11  0.033375  0.798613  0.030260  0.553965  0.040413   
[ParamStarve] 2-2   0.001327  0.741013  0.037402  0.500755  0.028279   
[ParamStarve] 2-3   0.019001  0.782066  0.020421  0.504565  0.016634   
[ParamStarve] 3-6   0.008875  0.785939  0.035170  0.531922  0.044170   
[ParamStarve] 1-2   0.012522  0.748336  0.007666  0.492837  0.004986   
[ParamStarve] 3-3   0.015274  0.710191  0.008777  0.474176  0.007903   
[ParamStarve] 3-2   0.008915  0.762804  0.046136  0.527512  0.042440   

category                                                                 \
item               best-val-acc-epoch            MESSIDOR-acc             
                                 mean        std         mean       std   
(run, subexp)                                                             
[ParamStarve] 2-22          47.666667   1.154701     0.512601  0.092772   
[ParamStarve] 2-6           46.666667   2.081666     0.622524  0.012209   
[ParamStarve] 2-11          45.666667   3.511885     0.566231  0.142906   
[ParamStarve] 1-11          37.666667   9.018500     0.499724  0.028365   
[ParamStarve] 1-22          38.666667   7.371115     0.485119  0.035816   
[ParamStarve] 1-6           28.333333   5.773503     0.566456  0.019221   
[ParamStarve] 3-22          45.666667   3.511885     0.574440  0.008419   
[ParamStarve] 3-11          45.000000   3.464102     0.547491  0.054160   
[ParamStarve] 2-2           45.000000   5.196152     0.545671  0.007150   
[ParamStarve] 2-3           45.666667   3.511885     0.511783  0.008001   
[ParamStarve] 3-6           46.333333   3.055050     0.564552  0.111868   
[ParamStarve] 1-2           38.000000  11.532563     0.489687  0.028962   
[ParamStarve] 3-3           45.000000   5.291503     0.505453  0.023374   
[ParamStarve] 3-2           44.666667   1.154701     0.589697  0.109120   

category                                               

In [10]:
d

category          run                             metrics                      \
item              exp   sub              subexp DRIVE-acc DRIVE-roc DRIVE-iou   
0         ParamStarve  2-22  [ParamStarve] 2-22  0.810259  0.903291  0.681035   
1         ParamStarve  2-22  [ParamStarve] 2-22  0.806113  0.891831  0.674725   
2         ParamStarve  2-22  [ParamStarve] 2-22  0.819517  0.900897  0.693700   
3         ParamStarve  2-22  [ParamStarve] 2-22  0.819834  0.910302  0.693776   
4         ParamStarve  2-22  [ParamStarve] 2-22  0.500835  0.498270  0.301153   
..                ...   ...                 ...       ...       ...       ...   
269       ParamStarve  3-22  [ParamStarve] 3-22  0.477929  0.500000  0.238965   
270       ParamStarve  3-22  [ParamStarve] 3-22  0.811872  0.900223  0.683185   
271       ParamStarve  3-22  [ParamStarve] 3-22  0.655893  0.714911  0.486479   
272       ParamStarve  3-22  [ParamStarve] 3-22  0.844525  0.921816  0.730881   
273       ParamStarve  3-22  [ParamStarve] 3-22  0.468182  0.490990  0.238366   

category                                                                   \
item       HRF-roc   HRF-iou best-val-acc-epoch MESSIDOR-acc MESSIDOR-roc   
0         0.840500  0.600222               33.0     0.635860     0.750224   
1         0.765150  0.541832               49.0     0.593972     0.664689   
2         0.800455  0.563230               41.0     0.626942     0.749821   
3         0.778202  0.540969               49.0     0.436968     0.511969   
4         0.507488  0.329032               16.0     0.390395     0.543185   
..             ...       ...                ...          ...          ...   
269       0.499900  0.239834               13.0     0.382979     0.500000   
270       0.812548  0.587452               46.0     0.566114     0.664708   
271       0.745262  0.518225               40.0     0.614115     0.684567   
272       0.846367  0.603219               49.0     0.574256     0.661944   
273       0.501501  0.243317               21.0     0.387489     0.497088   

category                         
item     MESSIDOR-iou   HRF-acc  
0            0.465677  0.751705  
1            0.422231  0.702901  
2            0.456293  0.721802  
3            0.266781  0.704074  
4            0.203277  0.498042  
..                ...       ...  
269          0.191490  0.479652  
270          0.390936  0.742016  
271          0.440932  0.682733  
272          0.402629  0.755480  
273          0.201221  0.481325  

[274 rows x 13 columns]